In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
path = './main_data/'
path2 = './main_data/events/'

In [3]:
organizations = pd.read_csv(path2 + 'organizations_updated.csv', sep=';')

In [4]:
organizations.head()

,ID,Учреждение,Адрес,ИНН,Категория,Код
0,5,ГБУК «Белгородский государственный историко-кр...,"Белгородская обл, г Белгород, ул Попова, д 2А",3.125017e+09,Музеи и галереи,31.0
1,6,ОГАУК «Драматический театр имени М. С. Щепкина»,"Белгородская обл, г Белгород, Соборная пл, д 1",3.124011e+09,Театры,31.0
2,7,ГБУК «Белгородский государственный литературны...,"Белгородская обл, г Белгород, ул Преображенска...",3.123310e+09,Музеи и галереи,31.0
3,9,ГБУК «Белгородский государственный центр народ...,"Белгородская обл, г Белгород, ул Широкая, д 1",3.124014e+09,Дворцы культуры и клубы,31.0
4,10,ГБУК «Белгородский государственный театр кукол»,"Белгородская обл, г Белгород, ул Некрасова, д ...",3.124012e+09,Театры,31.0


In [5]:
click = pd.read_csv(path + 'click.txt', sep=';', encoding= 'unicode_escape', error_bad_lines=False)

b'Skipping line 431497: expected 6 fields, saw 7\nSkipping line 431498: expected 6 fields, saw 7\n'
b'Skipping line 1071166: expected 6 fields, saw 7\nSkipping line 1071167: expected 6 fields, saw 7\n'
b'Skipping line 1422097: expected 6 fields, saw 7\n'
b'Skipping line 1537079: expected 6 fields, saw 7\n'


In [6]:
n_clicks_before = click.shape[0]
click = click.drop_duplicates(['create_time', 'user_id'])
n_clicks_after = click.shape[0]

print(f'Decreased # clicks from {n_clicks_before} to {n_clicks_after}')

Decreased # clicks from 1596723 to 1031660


In [7]:
click.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031660 entries, 0 to 1596722
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   create_time         1031660 non-null  object
 1   create_date         1031660 non-null  object
 2   user_phone_details  1031660 non-null  object
 3   buyer_mobile_phone  1031660 non-null  object
 4   user_id             1031660 non-null  object
 5   url                 1031660 non-null  object
dtypes: object(6)
memory usage: 55.1+ MB


In [8]:
events = pd.read_csv(path2 + 'events.csv', sep=';', error_bad_lines=False)

In [9]:
events = events.rename(columns= {'Ссылка на покупку билета': 'url',
                                 'ID': 'session_id',
                                 'Название события': 'session_name',
                                 'Дополнительная ссылка на покупку билета': 'additional_url',
                                 'Организатор мероприятия': 'organization_name'})

In [10]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42154 entries, 0 to 42153
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   session_id         42154 non-null  int64 
 1   session_name       42154 non-null  object
 2   organization_name  42154 non-null  object
 3   url                42151 non-null  object
 4   additional_url     3817 non-null   object
dtypes: int64(1), object(4)
memory usage: 1.6+ MB


In [11]:
events['session_identity'] = events['session_name'].apply(lambda x: x.split()[0].strip())

In [12]:
events = events[events['url'].notna()]

In [13]:
events.head()

,session_id,session_name,organization_name,url,additional_url,session_identity
0,36,Спектакль «Касатка»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль
1,40,Спектакль «Гамлет»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль
2,86,Спектакль «Горе от ума»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль
3,109,Спектакль «Женитьба»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль
4,139,Спектакль «Куклы»,ОГАУК «Драматический театр имени М. С. Щепкина»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,Спектакль


In [14]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42151 entries, 0 to 42153
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   session_id         42151 non-null  int64 
 1   session_name       42151 non-null  object
 2   organization_name  42151 non-null  object
 3   url                42151 non-null  object
 4   additional_url     3817 non-null   object
 5   session_identity   42151 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.3+ MB


In [15]:
events_pushka_accepted_30122021 = pd.read_csv(path2 + 'events_pushka_accepted_30122021.csv', sep = ',')

In [16]:
events_pushka_accepted_30122021.head(50)

,entity._id,entity.name,entity.saleLink,entity.additionalSaleLinks.0,entity.organization._id,entity.organization.name
0,36,Спектакль «Касатка»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
1,86,Спектакль «Горе от ума»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
2,86,Спектакль «Горе от ума»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
3,139,Спектакль «Куклы»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
4,150,Спектакль «Филумена Мартурано»,https://quicktickets.ru/belgorodskiy-dramatich...,NaN,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
5,155,Спектакль в фойе «С любовью не шутят»,https://quicktickets.ru/belgorodskiy-dramatich...,https://quicktickets.ru/belgorodskiy-dramatich...,6.0,ОГАУК «Драматический театр имени М. С. Щепкина»
6,890,Спектакль «Терешечка»,https://quicktickets.ru/belgorodskiy-gosudarst...,NaN,10.0,ГБУК «Белгородский государственный театр кукол»
7,1591,Спектакль «История солдата»,https://quicktickets.ru/belgorodskiy-gosudarst...,NaN,10.0,ГБУК «Белгородский государственный театр кукол»
8,1878,"Спектакль «Примадонны, или Шоу продолжается»",https://quicktickets.ru/teatrkaluga/e10,NaN,214.0,ГБУК КО «Калужский ордена Трудового Красного З...
9,1878,"Спектакль «Примадонны, или Шоу продолжается»",https://quicktickets.ru/teatrkaluga/e10,NaN,214.0,ГБУК КО «Калужский ордена Трудового Красного З...


In [17]:
events_pushka_accepted_30122021.loc[events_pushka_accepted_30122021['entity._id'] == 150999, 'entity.saleLink'] = 'https://tickets.tinchurinteatr.ru/event/1635'
events_pushka_accepted_30122021.loc[events_pushka_accepted_30122021['entity._id'] == 556682, 'entity.saleLink'] = 'https://tickets.tinchurinteatr.ru/event/1874'

In [18]:
events_pushka_accepted_30122021 = events_pushka_accepted_30122021[events_pushka_accepted_30122021['entity.saleLink'].notna()]

In [19]:
events_pushka_accepted_30122021['session_identity'] = events_pushka_accepted_30122021['entity.name'
                                                                                     ].apply(lambda x:
                                                                                            x.split()[0].strip())

In [20]:
events_pushka_accepted_30122021 = events_pushka_accepted_30122021.rename(columns= {'entity.saleLink': 'url',
                                                                                  'entity._id': 'session_id',
                                                                                  'entity.name': 'session_name',
                                                                                  'entity.additionalSaleLinks.0': 'additional_url',
                                                                                  'entity.organization._id': 'organization_id',
                                                                                  'entity.organization.name': 'organization_name'})

In [21]:
events_all = pd.concat([events_pushka_accepted_30122021, events])

In [22]:
events_all = events_all.drop_duplicates(['session_id', 'url', 'additional_url'])

In [23]:
events_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51092 entries, 0 to 42153
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   session_id         51092 non-null  int64  
 1   session_name       51092 non-null  object 
 2   url                51092 non-null  object 
 3   additional_url     5793 non-null   object 
 4   organization_id    35610 non-null  float64
 5   organization_name  51092 non-null  object 
 6   session_identity   51092 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 3.1+ MB


In [24]:
org_id_dict = dict(zip(organizations['Учреждение'], organizations['ID']))

In [25]:
from typing import List, Union, Optional

In [26]:
def get_org_id(org_name: str, org_dict: dict) -> Optional[int]:
    try:
        return org_dict[org_name]
    except KeyError:
        return None

In [27]:
events_all['organization_id'].fillna(events_all[events_all['organization_id'].isnull()]['organization_name'].apply(func=get_org_id, org_dict=org_id_dict), inplace=True)

In [28]:
events_all= events_all.astype({"organization_id": "int"})

In [29]:
events_all.tail(50)

,session_id,session_name,url,additional_url,organization_id,organization_name,session_identity
42104,1566274,«Фотоаппараты»,https://quicktickets.ru/karasuk-teatr-na-okrai...,NaN,26045,Государственное бюджетное учреждение культуры ...,«Фотоаппараты»
42105,1566298,"Концерт «Для вас, милые»",http://tickets.cdkis.ru/event/booking/2330,NaN,15986,МАУК «Ставропольский Дворец культуры и спорта»,Концерт
42106,1566299,«Про Иванушку-дурачка»,https://baldt.ru/bilety-on-line/#event/1162837,https://vk.com/app7010689_-11025454#events/116...,7891,МАУК «Балашовский драматический театр»,«Про
42107,1566302,Литературно-музыкальный спектакль «Легендарный...,https://novorosteatr.ru/repertoire/yanvar_2022...,https://novorosteatr.ru/repertoire/yanvar_2022...,11926,МАУ «Новороссийский городской театр»,Литературно-музыкальный
42108,1566313,Ленинград в блокаде,https://vmuzey.com/event/istoricheskiy-chas-le...,NaN,11960,МБУК «ЦБС» МР Белорецкий район,Ленинград
42109,1566323,«Шансон»,http://tickets.cdkis.ru/event/booking/2331,NaN,15986,МАУК «Ставропольский Дворец культуры и спорта»,«Шансон»
42110,1566331,«Поговори со мною о любви…»,https://vmuzey.com/event/koncert-pogovori-so-m...,NaN,10330,МБУ «Возрождение»,«Поговори
42111,1566332,Районный творческий конкурс среди мужчин Тазов...,https://tasuculture.ru/pay/,NaN,12770,МБУ «Централизованная сеть культурно-досуговых...,Районный
42112,1566343,Мастер-класс «Современная хореография»,https://xn--90aajgjchgadf7exa3o.xn--p1ai/afish...,NaN,11848,МАУ Бижбулякский ДК,Мастер-класс
42113,1566357,Спектакль «Василий Теркин»,https://ogatt.ru/buy-tickets/#/buy/4037,NaN,5556,БУК ОО «Орловский государственный академически...,Спектакль


In [30]:
events_1 = events_all[events_all['additional_url'].isnull()].drop('additional_url', axis=1)

In [31]:
events_2 = events_all[events_all['url'] == events_all['additional_url']].drop('additional_url', axis=1)

In [32]:
events_3 = events_all.loc[events_all['additional_url'].notna()].drop_duplicates(subset=['additional_url', 'url']).drop('additional_url', axis=1)

In [33]:
events_4 = events_all.loc[events_all['additional_url'].notna()].drop_duplicates(subset=['additional_url', 'url']).drop('url', axis=1).rename(columns = {'additional_url': 'url'})

In [34]:
event = pd.concat([events_1, events_2, events_3, events_4], ignore_index=True)

In [35]:
events_links = event['url'].unique()

In [36]:
len(events_links)

47736

In [37]:
click_links = click['url'].unique()

In [38]:
len(click_links)

19923

In [39]:
equal_links = []

for i in click_links:
    for j in events_links:
        if i == j:
            equal_links.append(i)

In [40]:
click_filter = click['url'].isin(equal_links)
click = click[click_filter]

In [41]:
diff = list(set(click_links) - set(equal_links))

In [42]:
len(diff)

166

In [43]:
cliks_add = pd.merge(click, event, how = 'left', on = 'url')

In [44]:
cliks_add.head()

,create_time,create_date,user_phone_details,buyer_mobile_phone,user_id,url,session_id,session_name,organization_id,organization_name,session_identity
0,2021-11-28 21:32:53,2021-11-28,"Android, 10 (SDK 29)",ce730a5e7bb6372aad126e934320b75c,08014dcd786881f349938c9255e9df36,https://theatreofnations.ru/performances/shved...,717865,Спектакль «Шведская спичка»,7577,ФГБУК «Государственный театр наций»,Спектакль
1,2021-11-28 21:41:52,2021-11-28,"Android, 10 (SDK 29)",ce730a5e7bb6372aad126e934320b75c,08014dcd786881f349938c9255e9df36,https://theatreofnations.ru/performances/shved...,717865,Спектакль «Шведская спичка»,7577,ФГБУК «Государственный театр наций»,Спектакль
2,2021-11-28 21:25:13,2021-11-28,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://tickets.hermitagemuseum.org/event/7E06...,1309020,Экспозиции главного музейного комплекса Эрмитажа,2217,ФГБУК «Государственный Эрмитаж»,Экспозиции
3,2021-11-28 22:11:33,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://tickets.kreml.ru/ru/#id=1&sid=763,1261967,Программа «Посещение архитектурного ансамбля С...,1848,Музеи Московского Кремля,Программа
4,2021-11-28 22:20:10,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://ramt.ru/plays/play-217/,1247322,Спектакль «Цветы для Элджернона»,4938,ФГБУК «Российский государственный академически...,Спектакль


In [45]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2661606 entries, 0 to 2661605
Data columns (total 11 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   create_time         object
 1   create_date         object
 2   user_phone_details  object
 3   buyer_mobile_phone  object
 4   user_id             object
 5   url                 object
 6   session_id          int64 
 7   session_name        object
 8   organization_id     int32 
 9   organization_name   object
 10  session_identity    object
dtypes: int32(1), int64(1), object(9)
memory usage: 233.5+ MB


In [46]:
cliks_add[cliks_add['session_identity'].isnull()]

,create_time,create_date,user_phone_details,buyer_mobile_phone,user_id,url,session_id,session_name,organization_id,organization_name,session_identity


In [47]:
organizations = organizations.rename(
    columns= {'ID': 'organization_id','Категория': 'org_category', 'Код': 'org_region_number'}) 

In [48]:
cliks_add = pd.merge(cliks_add, organizations[['organization_id', 'org_region_number', 'org_category']],
                     how = 'left', on = 'organization_id')

In [49]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2661696 entries, 0 to 2661695
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_time         object 
 1   create_date         object 
 2   user_phone_details  object 
 3   buyer_mobile_phone  object 
 4   user_id             object 
 5   url                 object 
 6   session_id          int64  
 7   session_name        object 
 8   organization_id     int32  
 9   organization_name   object 
 10  session_identity    object 
 11  org_region_number   float64
 12  org_category        object 
dtypes: float64(1), int32(1), int64(1), object(10)
memory usage: 274.1+ MB


In [50]:
cliks_add.head(10)

,create_time,create_date,user_phone_details,buyer_mobile_phone,user_id,url,session_id,session_name,organization_id,organization_name,session_identity,org_region_number,org_category
0,2021-11-28 21:32:53,2021-11-28,"Android, 10 (SDK 29)",ce730a5e7bb6372aad126e934320b75c,08014dcd786881f349938c9255e9df36,https://theatreofnations.ru/performances/shved...,717865,Спектакль «Шведская спичка»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры
1,2021-11-28 21:41:52,2021-11-28,"Android, 10 (SDK 29)",ce730a5e7bb6372aad126e934320b75c,08014dcd786881f349938c9255e9df36,https://theatreofnations.ru/performances/shved...,717865,Спектакль «Шведская спичка»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры
2,2021-11-28 21:25:13,2021-11-28,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://tickets.hermitagemuseum.org/event/7E06...,1309020,Экспозиции главного музейного комплекса Эрмитажа,2217,ФГБУК «Государственный Эрмитаж»,Экспозиции,78.0,Музеи и галереи
3,2021-11-28 22:11:33,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://tickets.kreml.ru/ru/#id=1&sid=763,1261967,Программа «Посещение архитектурного ансамбля С...,1848,Музеи Московского Кремля,Программа,77.0,Музеи и галереи
4,2021-11-28 22:20:10,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://ramt.ru/plays/play-217/,1247322,Спектакль «Цветы для Элджернона»,4938,ФГБУК «Российский государственный академически...,Спектакль,77.0,Театры
5,2021-11-28 22:23:07,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://teatr-sats.ru/nutcracker,80126,Балет «Щелкунчик»,4949,ФГБУК «Московский государственный академически...,Балет,77.0,Театры
6,2021-11-28 22:24:48,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://teatr-sats.ru/nutcracker,80126,Балет «Щелкунчик»,4949,ФГБУК «Московский государственный академически...,Балет,77.0,Театры
7,2021-11-28 22:28:29,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://theatreofnations.ru/performances/gorba...,752075,Спектакль «Горбачев»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры
8,2021-11-28 22:32:56,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://theatreofnations.ru/performances/gorba...,752075,Спектакль «Горбачев»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры
9,2021-11-28 22:45:46,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://theatreofnations.ru/performances/gorba...,752075,Спектакль «Горбачев»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры


In [51]:
cliks_add[cliks_add['org_category'].isnull()]['organization_name'].unique()

array(['Тестовое учреждение (встреча)', 'Александринский театр тестовое'],
      dtype=object)

In [52]:
cliks_add.loc[cliks_add['org_category'].isnull()]

,create_time,create_date,user_phone_details,buyer_mobile_phone,user_id,url,session_id,session_name,organization_id,organization_name,session_identity,org_region_number,org_category
2846,2021-11-28 20:36:09,2021-11-28,"iOS, iOS 15.0.2",05bcf16e627e30f0f4b9775adc33bf75,7516e97480a7745f0a23b913cecb120f,https://kazan-tuz.ru/repertoire/evening_116.html,1262561,Тестовое событие,28114,Тестовое учреждение (встреча),Тестовое,NaN,NaN
31403,2021-11-29 12:21:30,2021-11-29,"Android, 10 (SDK 29)",d12d285a5908d663f74fbceef3ea49cb,84811544d1dff4c85140a8e8879415f7,https://kazan-tuz.ru/repertoire/evening_116.html,1262561,Тестовое событие,28114,Тестовое учреждение (встреча),Тестовое,NaN,NaN
42175,2021-11-01 05:43:16,2021-11-01,"Android, 11 (SDK 30)",ced93f408a1a74ac2c4ce7d5a4ae50b7,e8a2e729b8ca7e86eca37728220c67b6,https://alexandrinsky.ru/afisha-i-bilety/chest...,1265995,Спектакль «Честная женщина» (тестовое),28114,Александринский театр тестовое,Спектакль,NaN,NaN
42176,2021-11-01 05:43:16,2021-11-01,"Android, 11 (SDK 30)",ced93f408a1a74ac2c4ce7d5a4ae50b7,e8a2e729b8ca7e86eca37728220c67b6,https://alexandrinsky.ru/afisha-i-bilety/chest...,1266129,Спектакль «Честная женщина» (тестовое 2),28114,Александринский театр тестовое,Спектакль,NaN,NaN
42177,2021-11-01 05:43:16,2021-11-01,"Android, 11 (SDK 30)",ced93f408a1a74ac2c4ce7d5a4ae50b7,e8a2e729b8ca7e86eca37728220c67b6,https://alexandrinsky.ru/afisha-i-bilety/chest...,1266225,Спектакль «Честная женщина» (тестовое),28114,Александринский театр тестовое,Спектакль,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2631937,2021-11-19 18:28:30,2021-11-19,"Android, 10 (SDK 29)",add237b16741c60d60f51073a801c751,70079464b00d632af8e785d57530397e,https://kazan-tuz.ru/repertoire/evening_116.html,1262561,Тестовое событие,28114,Тестовое учреждение (встреча),Тестовое,NaN,NaN
2631939,2021-11-19 18:38:30,2021-11-19,"Android, 10 (SDK 29)",add237b16741c60d60f51073a801c751,70079464b00d632af8e785d57530397e,https://kazan-tuz.ru/repertoire/evening_116.html,1262561,Тестовое событие,28114,Тестовое учреждение (встреча),Тестовое,NaN,NaN
2635949,2021-11-19 14:12:47,2021-11-19,"Android, 11 (SDK 30)",5900f24a7f09f40b057d1a707321c5d7,4e2ab89a79c1168d8c67f92159a5f2d7,https://kazan-tuz.ru/repertoire/evening_116.html,1262561,Тестовое событие,28114,Тестовое учреждение (встреча),Тестовое,NaN,NaN
2635951,2021-11-19 14:12:58,2021-11-19,"Android, 11 (SDK 30)",5900f24a7f09f40b057d1a707321c5d7,4e2ab89a79c1168d8c67f92159a5f2d7,https://kazan-tuz.ru/repertoire/evening_116.html,1262561,Тестовое событие,28114,Тестовое учреждение (встреча),Тестовое,NaN,NaN


In [53]:
cliks_add = cliks_add[cliks_add['org_category'].notna()]

In [54]:
cliks_add['type'] = cliks_add[['org_category', 'session_identity']].apply(lambda x: '_'.join(x), axis = 1)

In [74]:
cliks_add['type'].value_counts().head(30)

Театры_Спектакль                     959894
Театры_Балет                         109780
Музеи и галереи_Выставка             108131
Театры_Опера                          95401
Музеи и галереи_Экскурсия             54188
Театры_Концерт                        45519
Театры_Мюзикл                         38020
Концертные площадки_Концерт           31816
Музеи и галереи_Программа             23950
Музеи и галереи_Экспозиции            22377
Музеи и галереи_Мастер-класс          18365
Прочее_Спектакль                      17747
Театры_Оперетта                       16741
Театры_Программа                      16144
Музеи и галереи_Экспозиция            14317
Музеи и галереи_Лекция                14225
Прочее_Концерт                        11772
Театры_Музыкальная                    10409
Прочее_Программа                       9513
Дворцы культуры и клубы_Концерт        8708
Музеи и галереи_Обзорная               8116
Театры_Вечер                           7121
Прочее_Балет                    

In [55]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2661401 entries, 0 to 2661695
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_time         object 
 1   create_date         object 
 2   user_phone_details  object 
 3   buyer_mobile_phone  object 
 4   user_id             object 
 5   url                 object 
 6   session_id          int64  
 7   session_name        object 
 8   organization_id     int32  
 9   organization_name   object 
 10  session_identity    object 
 11  org_region_number   float64
 12  org_category        object 
 13  type                object 
dtypes: float64(1), int32(1), int64(1), object(11)
memory usage: 294.4+ MB


In [56]:
users = pd.read_csv(path + 'users_full.csv', sep=';')

In [57]:
users

,account_create_date,user_id,user_birth,app_open_date,user_phone_details,buyer_mobile_phone,user_phone_details_id,type_phone,region,age,user_region,diff,in_count
0,2021-09-02,ee457fb8cd024e46452147d026e3ffc4,2003-08-14,2021-11-22,"iOS, iOS 14.5",6a4816fe848efbd8c8fb8e8760d7cbd1,"iOS, 14",2,Саратовская область,18,64.0,81 days,1
1,2021-10-02,bb491c13ad78710345565c953cd82b7b,2000-03-06,2021-11-02,"Android, 11 (SDK 30)",527d5ae346fa42566f25ec418e6ece9c,"Android, 11",1,Новосибирская область,22,54.0,31 days,3
2,2021-11-28,13a12683c001f033edc9c7041283f728,2003-09-13,2021-11-28,"Android, 11 (SDK 30)",434c2b96b3dee8b5fa2a93cc3fec5013,"Android, 11",1,Архангельская область,18,29.0,0 days,1
3,2021-09-29,944c9e74b9b6e04be7e8cb867a66e709,2004-10-12,2021-11-25,"Android, 10 (SDK 29)",160e3490b932ca55fc133b11e74c100e,"Android, 10",1,Пермский край,17,59.0,57 days,3
4,2021-09-03,a36b45d2e722ebfb29287038ca473a2b,2004-01-24,2021-11-18,"iOS, iOS 14.2",920c30cd8c7dc4139e44fca6141c9b70,"iOS, 14",2,Москва,18,77.0,76 days,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205186,2021-11-22,f2c587cefd81ca1934230a2f9e0a3317,2000-05-26,2021-11-21,"iOS, iOS 15.0.2",67aa8fa9f2aa15df2a12d2f29c2085e4,"iOS, 15",2,NaN,22,NaN,-1 days,1
1205187,2021-09-01,15927678b718acf0232e6eb67969324d,2000-12-01,2021-11-05,"Android, 11 (SDK 30)",802cd1daf1e48edc4be593dfd8e08160,"Android, 11",1,Удмуртская Республика,21,18.0,65 days,1
1205188,2021-11-26,124719653c2ceb1b02b63c0e09b0c3b9,2004-02-12,2021-11-26,"Android, 8.0.0 (SDK 26)",d897cf53b90d24dce0a666721ba305d7,"Android, 8",1,NaN,18,NaN,0 days,1
1205189,2021-09-01,d5cf501ff24e7fcd2a73291ace60389c,2001-07-14,2021-11-03,"Android, 10 (SDK 29)",b9b81511da24efed598e0b0fae384e50,"Android, 10",1,Иркутская область,21,38.0,63 days,1


In [59]:
users = users[users['user_region'].notna()]

In [60]:
cliks_add = pd.merge(cliks_add, users[['user_id', 'user_region']],
                     how = 'left', on = 'user_id')

In [62]:
cliks_add.head(50)

,create_time,create_date,user_phone_details,buyer_mobile_phone,user_id,url,session_id,session_name,organization_id,organization_name,session_identity,org_region_number,org_category,type,user_region
0,2021-11-28 21:32:53,2021-11-28,"Android, 10 (SDK 29)",ce730a5e7bb6372aad126e934320b75c,08014dcd786881f349938c9255e9df36,https://theatreofnations.ru/performances/shved...,717865,Спектакль «Шведская спичка»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры,Театры_Спектакль,21.0
1,2021-11-28 21:41:52,2021-11-28,"Android, 10 (SDK 29)",ce730a5e7bb6372aad126e934320b75c,08014dcd786881f349938c9255e9df36,https://theatreofnations.ru/performances/shved...,717865,Спектакль «Шведская спичка»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры,Театры_Спектакль,21.0
2,2021-11-28 21:25:13,2021-11-28,"iOS, iOS 15.0.2",a1f60fad21d72c6dcb5fd7fe84fb3ad7,b2560eaa8bc23dbba5f24aa5075b89ed,https://tickets.hermitagemuseum.org/event/7E06...,1309020,Экспозиции главного музейного комплекса Эрмитажа,2217,ФГБУК «Государственный Эрмитаж»,Экспозиции,78.0,Музеи и галереи,Музеи и галереи_Экспозиции,77.0
3,2021-11-28 22:11:33,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://tickets.kreml.ru/ru/#id=1&sid=763,1261967,Программа «Посещение архитектурного ансамбля С...,1848,Музеи Московского Кремля,Программа,77.0,Музеи и галереи,Музеи и галереи_Программа,NaN
4,2021-11-28 22:20:10,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://ramt.ru/plays/play-217/,1247322,Спектакль «Цветы для Элджернона»,4938,ФГБУК «Российский государственный академически...,Спектакль,77.0,Театры,Театры_Спектакль,NaN
5,2021-11-28 22:23:07,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://teatr-sats.ru/nutcracker,80126,Балет «Щелкунчик»,4949,ФГБУК «Московский государственный академически...,Балет,77.0,Театры,Театры_Балет,NaN
6,2021-11-28 22:24:48,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://teatr-sats.ru/nutcracker,80126,Балет «Щелкунчик»,4949,ФГБУК «Московский государственный академически...,Балет,77.0,Театры,Театры_Балет,NaN
7,2021-11-28 22:28:29,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://theatreofnations.ru/performances/gorba...,752075,Спектакль «Горбачев»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры,Театры_Спектакль,NaN
8,2021-11-28 22:32:56,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://theatreofnations.ru/performances/gorba...,752075,Спектакль «Горбачев»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры,Театры_Спектакль,NaN
9,2021-11-28 22:45:46,2021-11-28,"Android, 12 (SDK 31)",a352caeadc9ab5e2dd4e8dfd9913e02e,fdfe16d86915392aeeca341505cce1c1,https://theatreofnations.ru/performances/gorba...,752075,Спектакль «Горбачев»,7577,ФГБУК «Государственный театр наций»,Спектакль,77.0,Театры,Театры_Спектакль,NaN


In [63]:
cliks_add = cliks_add[cliks_add['user_region'].notna()]

In [64]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1906492 entries, 0 to 2661400
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_time         object 
 1   create_date         object 
 2   user_phone_details  object 
 3   buyer_mobile_phone  object 
 4   user_id             object 
 5   url                 object 
 6   session_id          int64  
 7   session_name        object 
 8   organization_id     int32  
 9   organization_name   object 
 10  session_identity    object 
 11  org_region_number   float64
 12  org_category        object 
 13  type                object 
 14  user_region         float64
dtypes: float64(2), int32(1), int64(1), object(11)
memory usage: 225.5+ MB


In [67]:
short_cliks = cliks_add.drop_duplicates(subset=['user_id', 'session_id'])

In [68]:
short_cliks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041726 entries, 0 to 2661395
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   create_time         1041726 non-null  object 
 1   create_date         1041726 non-null  object 
 2   user_phone_details  1041726 non-null  object 
 3   buyer_mobile_phone  1041726 non-null  object 
 4   user_id             1041726 non-null  object 
 5   url                 1041726 non-null  object 
 6   session_id          1041726 non-null  int64  
 7   session_name        1041726 non-null  object 
 8   organization_id     1041726 non-null  int32  
 9   organization_name   1041726 non-null  object 
 10  session_identity    1041726 non-null  object 
 11  org_region_number   1041726 non-null  float64
 12  org_category        1041726 non-null  object 
 13  type                1041726 non-null  object 
 14  user_region         1041726 non-null  float64
dtypes: float64(2), 

In [70]:
short_cliks[short_cliks['org_region_number'] == short_cliks['user_region']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 825389 entries, 10 to 2661395
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   create_time         825389 non-null  object 
 1   create_date         825389 non-null  object 
 2   user_phone_details  825389 non-null  object 
 3   buyer_mobile_phone  825389 non-null  object 
 4   user_id             825389 non-null  object 
 5   url                 825389 non-null  object 
 6   session_id          825389 non-null  int64  
 7   session_name        825389 non-null  object 
 8   organization_id     825389 non-null  int32  
 9   organization_name   825389 non-null  object 
 10  session_identity    825389 non-null  object 
 11  org_region_number   825389 non-null  float64
 12  org_category        825389 non-null  object 
 13  type                825389 non-null  object 
 14  user_region         825389 non-null  float64
dtypes: float64(2), int32(1), int64(1

In [71]:
cliks_add.to_csv(path + 'cliks_add_2.csv', sep=';', index=False)